# Analyzing Borrowers’ Risk of Defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit score** of a potential customer. A **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
## Project: Data Processing

#Import data
import pandas as pd

try:
    credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')
except:
    print('Something went wrong')
    
#General information about data.
credit_scoring.info()
#Last ten rows of data
credit_scoring.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,secondary education,1,married,0,F,employee,1,17517.812,to become educated
21516,0,-914.391429,42,bachelor's degree,0,married,0,F,business,0,51649.244,purchase of my own house
21517,0,-404.679034,42,bachelor's degree,0,civil partnership,1,F,business,0,28489.529,buying my own car
21518,0,373995.710838,59,SECONDARY EDUCATION,1,married,0,F,retiree,0,24618.344,purchase of a car
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


### Conclusion

The goal of this project is to find the best clients for repaying a loan. My plan is to compare the ratio of a persons bad and good debt in regards to family status, children, total income, and purpose. By doing so, it will help the company understand which clientel would be best suited for approval of a loan. 

By looking at the information in the data we can see that there are 12 columns and 21525 rows. The data consists of int64, float64, and object types. Values are missing from both the days_employed and total_income columns, since they have different non-null values.

## Data preprocessing

### Processing missing values

In [2]:
#Locate missing values per column.
credit_scoring.isnull().sum()

#Percentage of missing values in every column
credit_scoring.isnull().sum()*100/len(credit_scoring)

#Change all values in education to lower case
credit_scoring['education'] = credit_scoring['education'].str.lower()

#Get rid of error value '20' in column children
credit_scoring['children'].unique()
credit_scoring = credit_scoring.loc[credit_scoring["children"] != 20]

#Get rid of errror value 'XNA' in column gender
credit_scoring['gender'].unique()
credit_scoring = credit_scoring.loc[credit_scoring["gender"] != 'XNA']

#Median of total_income grouped by income_type and education
credit_scoring.groupby(['income_type', 'education'])['total_income'].median()

#Fill in the missing values with the group medians of total_income grouped by income_type and education.
credit_scoring['total_income'] = credit_scoring.groupby(['income_type', 'education'])['total_income'].apply(lambda x: x.fillna(x.median()))

#Median of days_employed grouped by income_type and education.
credit_scoring.groupby(['income_type','education'])['days_employed'].median()

#Absolute value of column days_employed to remove negative values
credit_scoring['days_employed'] = credit_scoring['days_employed'].abs()

#Fill in the missing values with the group median of days_employed grouped by income_type and education.
credit_scoring['days_employed'] = credit_scoring.groupby(['income_type', 'education'])['days_employed'].apply(lambda x: x.fillna(x.median()))

#Check to make sure missing values are filled in
credit_scoring.isnull().sum()


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Conclusion**

The type of missing values are integers, in the total_income and days_employed column. The missing values were the same rows of both columns. It can be that the total_income column is dependent on the days_employed column. If the days_employed data is missing then the total_income for the person would be missing as well. The missing data in the days_employed column is missing from human error.

I replaced the missing data in the column total_income by finding the group medians of total_income grouped by income_type and education. By doing it this way the numbers replacing the missing values will be closer to the actual total income value.

### Data type replacement

In [3]:
#Replace float type with int type in columns total_income and days_employed
credit_scoring['total_income'] = pd.to_numeric(credit_scoring['total_income']).astype('int')
credit_scoring['days_employed'] = pd.to_numeric(credit_scoring['days_employed']).astype('int')

**Conclusion**

I changed both float values with integers values. This gets rid of the decimals and makes the data cleaner.

### Processing duplicates

In [4]:
#How many duplicated values in data.
credit_scoring.duplicated().sum()
#Drop duplicates.
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop = True)
#Check duplicates are dropped and index is reset.
credit_scoring.duplicated().sum()

0

**Conclusion**

The number of duplicate values is 71. There is only 0.3% of duplicate data. This is an unsignificant amount so I can delete data. The reason for duplicates can be caused by human error.

### Categorizing Data

In [5]:
credit_scoring.groupby(['income_type', 'education'])['total_income', 'debt'].median()

total_income  debt
income_type                 education                              
business                    bachelor's degree           32305     0
                            primary education           21887     0
                            secondary education         25458     0
                            some college                28778     0
civil servant               bachelor's degree           27601     0
                            graduate degree             17822     0
                            primary education           23734     0
                            secondary education         21856     0
                            some college                25694     0
employee                    bachelor's degree           26521     0
                            graduate degree             31771     0
                            primary education           20159     0
                            secondary education         21843     0
                            some college                24260     0
entrepreneur                bachelor's degree           79866     0
paternity / maternity leave secondary education          8612     1
retiree                     bachelor's degree           23078     0
                            graduate degree             28334     0
                            primary education           16415     0
                            secondary education         18370     0
                            some college                19221     0
student                     bachelor's degree           15712     0
unemployed                  bachelor's degree           32435     0
                            secondary education          9593     1

**Conclusion**

I categorized my data by income_type and education with total_income and debt. I decided to categorize this ways because I think that total_income and debt are important when deciding customer credit worthiness. I grouped by income_type and education to see the data better.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [6]:
#Absolute value to make the row with -1 children positive.
credit_scoring['children'] = credit_scoring['children'].abs()

#A function titled children_debt to obtain good or bad debt status depending on amount of children.
def children_debt(row):
    children = row['children']
    debt = row['debt']
    
    if children == 0:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if children == 1:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if children == 2:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if children == 3:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if children == 4:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if children == 5:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if children == 20:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    
#Add the function children_debt to the table.
credit_scoring['children_debt_group'] = credit_scoring.apply(children_debt, axis = 1)
#Create a pivot table with data on children and debt status.
data_pivot_children = credit_scoring.pivot_table(
    index = 'children', columns = 'children_debt_group', values = 'debt', aggfunc = 'count'
)

#Drop row with 5 children since there are missing values. 
#I removed this row because there was only 7 cases total and no bad debt among families with 5 children.
data_pivot_children = data_pivot_children.dropna()
#Add a column for ratio of bad to good debt.
data_pivot_children['ratio_debt'] = data_pivot_children['bad_debt_status'] / data_pivot_children['good_debt_status']
data_pivot_children.head(10)

children_debt_group,bad_debt_status,good_debt_status,ratio_debt
children,,,
0,1063.0,13027.0,0.081600
1,445.0,4410.0,0.100907
2,194.0,1858.0,0.104413
3,27.0,303.0,0.089109
4,4.0,37.0,0.108108


**Conclusion**

Clients with no children have a smaller bad to good ratio than customers that have children. A client is more likely to repay a loan if they do not have children.

- Is there a relation between marital status and repaying a loan on time?

In [7]:
#A function to calculate good and bad debt based on family status
def family_status_debt(row):
    family_status_id = row['family_status_id']
    debt = row['debt']
    
    if family_status_id == 0:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if family_status_id == 1:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if family_status_id == 2:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if family_status_id == 3:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if family_status_id == 4:
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'

#Apply our function to the table.
credit_scoring['family_status_debt_group'] = credit_scoring.apply(family_status_debt, axis = 1)
#Create a pivot table with data on family status and debt status.
data_pivot_family_status = credit_scoring.pivot_table(
    index = 'family_status', columns = 'family_status_debt_group', values = 'debt', aggfunc = 'count'
)
#Add a column to pivot table with ratio of bad to good debt.
data_pivot_family_status['ratio_debt'] = data_pivot_family_status['bad_debt_status'] / data_pivot_family_status['good_debt_status']

#The mean and median values of ratio debt.
data_pivot_family_status['ratio_debt'].mean()
data_pivot_family_status['ratio_debt'].median()

data_pivot_family_status.head()


family_status_debt_group,bad_debt_status,good_debt_status,ratio_debt
family_status,,,
civil partnership,385,3753,0.102585
divorced,84,1109,0.075744
married,928,11362,0.081676
unmarried,273,2528,0.107991
widow / widower,63,892,0.070628


**Conclusion**

The bad to good ratio of debt is higher for civil partnerships and unmarried clients. It is more likely that divorced, married, or widowed customers will not default on a loan.

- Is there a relation between income level and repaying a loan on time?

In [8]:
def total_income_status(row):
    total_income = row['total_income']
    
    if total_income <= 100000:
        return 'low income'
    if 100000 < total_income <= 200000:
        return 'medium income'
    return 'high income'
    
credit_scoring['total_income_status'] = credit_scoring.apply(total_income_status, axis = 1)
    
#A function to calculate good and bad debt status based on total income.
def total_income_debt(row):
    debt = row['debt']
    
    if debt == 0:
        return 'good_debt_status'
    return 'bad_debt_status'
    
#Apply the function to the data table.
credit_scoring['total_income_debt_group'] = credit_scoring.apply(total_income_debt, axis = 1)
#Create a pivot table with data on debt status and total income
data_pivot_total_income = credit_scoring.pivot_table(
    index = ['total_income_status'], columns = 'total_income_debt_group', values = 'debt', aggfunc = 'count'
)
#Create the column ratio to calculate the ratio of bad debt to good debt.
data_pivot_total_income['ratio_debt'] = (
    data_pivot_total_income['bad_debt_status'] / data_pivot_total_income['good_debt_status']
)
#Show debt status for the highest and lowest total income values.
data_pivot_total_income.head()

total_income_debt_group,bad_debt_status,good_debt_status,ratio_debt
total_income_status,,,
high income,1,10,0.100000
low income,1727,19551,0.088333
medium income,5,83,0.060241


**Conclusion**

Based on the data customers that have a total income between 100,000 and 200,000 are the least likely to default on a loan. Customers with high income are the most likely to default on a loan.

- How do different loan purposes affect on-time repayment of the loan?

In [9]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter

#The unique values in the purpose column
credit_scoring['purpose'].unique()

english_stemmer = SnowballStemmer('english')
wordnet_lemma = WordNetLemmatizer()

#A function for categorizing the purpose column using stemming
def purpose_categorize(purpose):
    words = nltk.word_tokenize(purpose)
    stems = [english_stemmer.stem(w) for w in words if w.isalpha()]
    
    if 'wed' in stems:
        return 'wedding'
    elif 'car' in stems:
        return 'cars'
    elif ('univers' in stems) or ('educ' in stems):
        return 'education'
    elif ('real est' in stems) or ('properti' in stems) or ('hous'):
        return 'housing'
        
#Applying the function to our data
credit_scoring['purpose_id'] = credit_scoring['purpose'].apply(lambda x: purpose_categorize(x))
credit_scoring.head()

#Function to calculate debt status based on loan purpose.
def purpose_debt(row):
    purpose_id = row['purpose_id']
    debt = row['debt']
    
    if purpose_id == 'wedding':
        if debt == 0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if purpose_id == 'cars':
        if debt ==0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if purpose_id == 'education':
        if debt ==0:
            return 'good_debt_status'
        return 'bad_debt_status'
    if purpose_id == 'housing':
        if debt ==0:
            return 'good_debt_status'
        return 'bad_debt_status'
    
#Add the function to the data table.    
credit_scoring['purpose_debt_group'] = credit_scoring.apply(purpose_debt, axis = 1)
#Create a pivot table with loan purpose and debt status.
data_pivot_purpose = credit_scoring.pivot_table(
    index = ['purpose_id'], columns = 'purpose_debt_group', values = 'debt', aggfunc = 'count'
)
#Create column ratio to calculate the ratio of bad debt to good debt.
data_pivot_purpose['ratio'] = data_pivot_purpose['bad_debt_status'] / data_pivot_purpose['good_debt_status']
data_pivot_purpose.head(50)


purpose_debt_group,bad_debt_status,good_debt_status,ratio
purpose_id,,,
cars,401,3889,0.103111
education,369,3629,0.101681
housing,780,9994,0.078047
wedding,183,2132,0.085835


**Conclusion**

We can see from the data that clients obtaining a loan for the purpose of cars and education are more likely to default on a loan. Where as, customers applying for a loan with the purpose of budgeting a wedding and a house are less likely to default on a loan.

## General Conclusion

Based on the data, the best clients to consider for a loan are divorced, married, or widowed with no children. Customers with a medium income base, are the least likely to default on a loan. Clients applying for loans with the purpose of buying a house or money for a wedding are the least likely to default on a loan.

## Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [ ]  file examined;
- [ ]  missing values defined;
- [ ]  missing values are filled;
- [ ]  an explanation of which missing value types were detected;
- [ ]  explanation for the possible causes of missing values;
- [ ]  an explanation of how the blanks are filled;
- [ ]  replaced the real data type with an integer;
- [ ]  an explanation of which method is used to change the data type and why;
- [ ]  duplicates deleted;
- [ ]  an explanation of which method is used to find and remove duplicates;
- [ ]  description of the possible reasons for the appearance of duplicates in the data;
- [ ]  data is categorized;
- [ ]  an explanation of the principle of data categorization;
- [ ]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [ ]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [ ]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [ ]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [ ]  conclusions are present on each stage;
- [ ]  a general conclusion is made.